# Bounding Box Visualizer

In [1]:
COLORS = [ 
    "#6793be", "#990000", "#00ff00", "#ffbcc9", "#ffb9c7", "#fdc6d1",
    "#fdc9d3", "#6793be", "#73a4d4", "#9abde0", "#9abde0", "#8fff8f", "#ffcfd8", "#808080", "#808080",
    "#ffba00", "#6699ff", "#009933", "#1c1c1c", "#08375f", "#116ebf", "#e61d35", "#106bff", "#8f8fff",
    "#8fff8f", "#dbdbff", "#dbffdb", "#dbffff", "#ffdbdb", "#ffc2c2", "#ffa8a8", "#ff8f8f", "#e85e68",
    "#123456", "#5cd38c", "#1d1f5f", "#4e4b04", "#495a5b", "#489d73", "#9d4872", "#d49ea6", "#ff0080",
    "#6793be", "#990000", "#fececf", "#ffbcc9", "#ffb9c7", "#fdc6d1",
    "#fdc9d3", "#6793be", "#73a4d4", "#9abde0", "#9abde0", "#8fff8f", "#ffcfd8", "#808080", "#808080",
    "#ffba00", "#6699ff", "#009933", "#1c1c1c", "#08375f", "#116ebf", "#e61d35", "#106bff", "#8f8fff",
    "#8fff8f", "#dbdbff", "#dbffdb", "#dbffff", "#ffdbdb", "#ffc2c2", "#ffa8a8", "#ff8f8f", "#e85e68",
    "#123456", "#5cd38c", "#1d1f5f", "#4e4b04", "#495a5b", "#489d73", "#9d4872", "#d49ea6", "#ff0080" 
]

In [2]:
try:
    import cv2
except ImportError:
    cv2 = None

def annotate_image(image, detection):
    """ Annotate images with object detection results
    # Arguments:
        image: numpy array representing the image used for detection
        detection: `DetectionResult` result from SKIL on the same image
    # Return value:
        annotated image as numpy array
    """
    if cv2 is None:
        raise Exception("OpenCV is not installed.")
    objects = detection.get('objects')
    if objects:
        for detect in objects:
            confs = detect.get('confidences')
            max_conf = max(confs)
            max_index = confs.index(max_conf)
            classes = detect.get('predictedClasses')
            max_class = classes[max_index]
            h = detect.get('height')
            w = detect.get('width')
            center_x = detect.get('centerX')
            center_y = detect.get('centerY')
            lower = (int(center_x - w/2), int(center_y - h/2))
            upper = (int(center_x + w/2), int(center_y + h/2))
            im = cv2.rectangle(image, lower, upper, (255, 0, 0), 3)
            cv2.putText(image, max_class + " " + str(max_conf),
                        upper, cv2.FONT_HERSHEY_SIMPLEX, 1, 0, 2)
    return image